In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model, Model

In [3]:
def predict_violence(video_path, model_path):
    # Load the trained LSTM model
    model = load_model(model_path)

    # Define constants
    img_size = 224
    num_channels = 3
    frames_per_video = 20
    chunk_size = 4096
    n_chunks = 20

    # Preprocess the video frames
    frames = []
    cap = cv2.VideoCapture(video_path)
    while len(frames) < frames_per_video:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        frame = frame / 255.0
        frames.append(frame)
    cap.release()

    # Convert frames to numpy array
    frames = np.array(frames)

    # Load the saved VGG16 model for feature extraction
    vgg16_model = load_model('vgg16_model.h5')
    transfer_layer = vgg16_model.get_layer('fc2')
    vgg16_model_transfer = Model(inputs=vgg16_model.input, outputs=transfer_layer.output)

    # Extract transfer values from VGG16 model
    transfer_values = vgg16_model_transfer.predict(frames)

    # Reshape transfer values to match LSTM input shape
    transfer_values = transfer_values.reshape((1, n_chunks, chunk_size))

    # Make prediction using the LSTM model
    predictions = model.predict(transfer_values)

    # Classify the video as violence or non-violence based on the prediction
    
    if predictions[0][0] > predictions[0][1]:
        return "Violence detected"
    else:
        return "No violence detected"

In [6]:
video_path = "/Users/ricks/Internship/DWDM/vcheck4.mp4"
model_path = "model.h5"
result = predict_violence(video_path, model_path)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
No violence detected
